# Assignment 9: GBDT

#### Response Coding: Example

<img src='http://i.imgur.com/TufZptV.jpg' width=700px>

> The response tabel is built only on train dataset.
> For a category which is not there in train data and present in test data, we will encode them with default values
Ex: in our test data if have State: D then we encode it as [0.5, 0.05]

<ol>
    <li><strong>Apply GBDT on these feature sets</strong>
        <ul>
            <li><font color='red'>Set 1</font>: categorical(instead of one hot encoding, try <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/handling-categorical-and-numerical-features/'>response coding</a>: use probability values), numerical features + project_title(TFIDF)+  preprocessed_eassay (TFIDF)+sentiment Score of eassay(check the bellow example, include all 4 values as 4 features)</li>
            <li><font color='red'>Set 2</font>: categorical(instead of one hot encoding, try <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/handling-categorical-and-numerical-features/'>response coding</a>: use probability values), numerical features + project_title(TFIDF W2V)+  preprocessed_eassay (TFIDF W2V)</li>        </ul>
    </li>
    <li><strong>The hyper paramter tuning (Consider any two hyper parameters)</strong>
        <ul>
    <li>Find the best hyper parameter which will give the maximum <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/receiver-operating-characteristic-curve-roc-curve-and-auc-1/'>AUC</a> value</li>
    <li>find the best hyper paramter using k-fold cross validation/simple cross validation data</li>
    <li>use gridsearch cv or randomsearch cv or you can write your own for loops to do this task</li>
        </ul>
    </li>
    <li>
    <strong>Representation of results</strong>
        <ul>
    <li>You need to plot the performance of model both on train data and cross validation data for each hyper parameter, like shown in the figure
    <img src='https://i.imgur.com/Gp2DQmh.jpg' width=500px> with X-axis as <strong>n_estimators</strong>, Y-axis as <strong>max_depth</strong>, and Z-axis as <strong>AUC Score</strong> , we have given the notebook which explains how to plot this 3d plot, you can find it in the same drive <i>3d_scatter_plot.ipynb</i></li>
            <p style="text-align:center;font-size:30px;color:red;"><strong>or</strong></p> <br>
    <li>You need to plot the performance of model both on train data and cross validation data for each hyper parameter, like shown in the figure
    <img src='https://i.imgur.com/fgN9aUP.jpg' width=300px> <a href='https://seaborn.pydata.org/generated/seaborn.heatmap.html'>seaborn heat maps</a> with rows as <strong>n_estimators</strong>, columns as <strong>max_depth</strong>, and values inside the cell representing <strong>AUC Score</strong> </li>
    <li>You choose either of the plotting techniques out of 3d plot or heat map</li>
    <li>Once after you found the best hyper parameter, you need to train your model with it, and find the AUC on test data and plot the ROC curve on both train and test.
    <img src='https://i.imgur.com/wMQDTFe.jpg' width=300px></li>
    <li>Along with plotting ROC curve, you need to print the <a href='https://www.appliedaicourse.com/course/applied-ai-course-online/lessons/confusion-matrix-tpr-fpr-fnr-tnr-1/'>confusion matrix</a> with predicted and original labels of test data points
    <img src='https://i.imgur.com/IdN5Ctv.png' width=300px></li>
            </ul>
    <br>
    <li>You need to summarize the results at the end of the notebook, summarize it in the table format
        <img src='http://i.imgur.com/YVpIGGE.jpg' width=400px>
    </li>
</ol>

In [0]:
!pip install category_encoders

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from google.colab import files
from tqdm import tqdm

from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer

from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.externals import joblib
from sklearn.metrics import roc_curve, auc, accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
from category_encoders import TargetEncoder

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


ModuleNotFoundError: ignored

In [0]:
#Mounting drive work folder
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/My Drive/Appliedai colab/Assignment 11 - GBDT donors choose/

Mounted at /content/drive
/content/drive/My Drive/Appliedai colab/Assignment 11 - GBDT donors choose


<h1>1. GBDT (xgboost/lightgbm/catboost) </h1>

## 1.1 Loading Data

In [0]:
data = pd.read_csv('preprocessed_data.csv', nrows = 50000)
train = pd.read_csv('train_data.csv')

In [0]:
#https://scikit-learn.org/stable/auto_examples/compose/plot_column_transformer_mixed_types.html
#https://stackoverflow.com/a/54704747/9292995

def Donors_choose_pipe(data, model, text_transformer):
  numeric_features = ['teacher_number_of_previously_posted_projects', 'price']
  numeric_transformer = Pipeline(steps=[
      ('scaler', MinMaxScaler())])

  categorical_features = ['school_state', 'teacher_prefix', 'project_grade_category','clean_categories','clean_subcategories']
  categorical_transformer = Pipeline(steps=[
      ('resp', TargetEncoder())])

  # text_features = ['essay','project_title']
  text_transformer = Pipeline(steps=[
      ('trans', text_transformer)])

  preprocessor = ColumnTransformer(
      transformers=[('num', numeric_transformer, numeric_features),
          ('cat', categorical_transformer, categorical_features),
          ('essay', text_transformer, "essay"),
          # ('title', text_transformer, "project_title")
          ]
      ,n_jobs=-1, verbose=True, remainder = 'passthrough'
              )

  # Append classifier to preprocessing pipeline.
  # Now we have a full prediction pipeline.
  pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier',  model)])

  y = data['project_is_approved'].values
  X = data.drop(['project_is_approved'], axis=1)

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, stratify=y)

  pipeline.fit(X_train, y_train)
  print("model score: %.3f" % pipeline.score(X_test, y_test))
  return pipeline, X, y, X_train, X_test, y_train, y_test

In [0]:
def searchplot(rs, alpha, title):
  print(("best from search: %.3f"
        % rs.score(X_test, y_test)))

  results = pd.DataFrame.from_dict(rs.cv_results_)

  results = results.sort_values(alpha)
  train_auc= results['mean_train_score']
  train_auc_std= results['std_train_score']
  cv_auc = results['mean_test_score'] 
  cv_auc_std= results['std_test_score']

  K =  results[alpha]

  plt.plot(K, train_auc, label='Train AUC')
  # https://stackoverflow.com/a/48803361/4084039
  # plt.gca().fill_between(K, train_auc - train_auc_std,train_auc + train_auc_std,alpha=0.2,color='darkblue')

  plt.plot(K, cv_auc, label='CV AUC')

  plt.scatter(K, train_auc, label='Train AUC points')
  plt.scatter(K, cv_auc, label='CV AUC points')


  plt.legend()
  plt.xscale('log')
  plt.xlabel("Alpha")
  plt.ylabel("AUC")
  plt.title(title)
  plt.grid()
  plt.show()

  results.head()

In [0]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html#sklearn.metrics.roc_curve

def roc_auc_pipe(clf, X_train, X_test, y_train, y_test, title):
  y_train_pred = clf.predict_proba(X_train)[:,1]
  y_test_pred = clf.predict_proba(X_test)[:,1]
      
  train_fpr, train_tpr, tr_thresholds = roc_curve(y_train, y_train_pred)
  test_fpr, test_tpr, te_thresholds = roc_curve(y_test, y_test_pred)

  plt.close
  plt.plot(train_fpr, train_tpr, label="train AUC ="+str(auc(train_fpr, train_tpr)))
  plt.plot(test_fpr, test_tpr, label="test AUC ="+str(auc(test_fpr, test_tpr)))
  plt.legend()
  plt.plot([0, 1], [0, 1],'g--')
  plt.xlim([0, 1])
  plt.ylim([0, 1])
  plt.xlabel("False Positive Rate(FPR)")
  plt.ylabel("True Positive Rate(TPR)")
  plt.title(title)
  plt.grid()
  plt.show()

Set 1: categorical(instead of one hot encoding, try response coding: use probability values), numerical features + project_title(TFIDF)+ preprocessed_eassay (TFIDF)+sentiment Score of eassay(check the bellow example, include all 4 values as 4 features)

In [0]:
%%time
seed = 42
text_transformer = TfidfVectorizer(stop_words = 'english', min_df = 10)
model = XGBClassifier(tree_method = "gpu_hist", 
                      random_state=seed, 
                      eval_metric=["auc"],
                      verbosity = 2)
clf, X, y, X_train, X_test, y_train, y_test = Donors_choose_pipe(data, model, text_transformer)

model score: 0.841
CPU times: user 7.33 s, sys: 4.77 s, total: 12.1 s
Wall time: 20.4 s


In [0]:
clf.save_model('XGB TFIDF.model')

NameError: ignored

In [0]:
print("model score: %.3f" % clf.score(X_test, y_test))

model score: 0.841


In [0]:
%%time
param_grid = {
    'classifier__alpha': [0.001,0.01,0.1,1,10,100],
    'classifier__max_depth': [7],
    'classifier__n_estimators': [300]
}
#adding n_jobs = -1 causes memory leaks
rs = RandomizedSearchCV(clf, param_grid, cv = 3, n_jobs = -1,  verbose=15, return_train_score = True, scoring = "roc_auc", refit = True)

rs.fit(X_train, y_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  2.1min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  2.1min finished


CPU times: user 26.6 s, sys: 18.7 s, total: 45.3 s
Wall time: 2min 56s


In [0]:
# specify parameters and distributions to sample from
from sklearn.utils.fixes import loguniform
param_dist = {'average': [True, False],
              'l1_ratio': stats.uniform(0, 1),
              'alpha': loguniform(1e-4, 1e0)}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

In [0]:
# Utility function to report best scores
from time import time
start = time()
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.8f} (std: {1:.8f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(rs.cv_results_['params'])))
report(rs.cv_results_)

GridSearchCV took 0.00 seconds for 1 candidate parameter settings.
Model with rank: 1
Mean validation score: 0.71014131 (std: 0.00212398)
Parameters: {'classifier__n_estimators': 300, 'classifier__max_depth': 7, 'classifier__alpha': 0.1}



In [0]:
rs.best_params_, rs.best_score_

({'classifier__alpha': 0.1,
  'classifier__max_depth': 7,
  'classifier__n_estimators': 300},
 0.7103543010541454)

In [0]:
sorted(rs.cv_results_['params'], key = lambda i: (i['classifier__alpha'], i['classifier__max_depth'])) 

[{'classifier__alpha': 0.001,
  'classifier__max_depth': 7,
  'classifier__n_estimators': 100},
 {'classifier__alpha': 0.001,
  'classifier__max_depth': 7,
  'classifier__n_estimators': 200},
 {'classifier__alpha': 0.001,
  'classifier__max_depth': 7,
  'classifier__n_estimators': 300},
 {'classifier__alpha': 0.001,
  'classifier__max_depth': 7,
  'classifier__n_estimators': 400},
 {'classifier__alpha': 0.001,
  'classifier__max_depth': 7,
  'classifier__n_estimators': 500},
 {'classifier__alpha': 0.001,
  'classifier__max_depth': 8,
  'classifier__n_estimators': 100},
 {'classifier__alpha': 0.001,
  'classifier__max_depth': 8,
  'classifier__n_estimators': 200},
 {'classifier__alpha': 0.001,
  'classifier__max_depth': 8,
  'classifier__n_estimators': 300},
 {'classifier__alpha': 0.001,
  'classifier__max_depth': 8,
  'classifier__n_estimators': 400},
 {'classifier__alpha': 0.001,
  'classifier__max_depth': 8,
  'classifier__n_estimators': 500}]

In [0]:
for param, test_score in zip(rs.cv_results_['params'], rs.cv_results_['mean_test_score']):
  print(param, "score:", test_score)

{'classifier__n_estimators': 100, 'classifier__max_depth': 7, 'classifier__alpha': 0.001} score: 0.7034139633834661
{'classifier__n_estimators': 200, 'classifier__max_depth': 7, 'classifier__alpha': 0.001} score: 0.7078884532939395
{'classifier__n_estimators': 300, 'classifier__max_depth': 7, 'classifier__alpha': 0.001} score: 0.709007574895249
{'classifier__n_estimators': 400, 'classifier__max_depth': 7, 'classifier__alpha': 0.001} score: 0.7075459051710401
{'classifier__n_estimators': 500, 'classifier__max_depth': 7, 'classifier__alpha': 0.001} score: 0.7073377493468471
{'classifier__n_estimators': 100, 'classifier__max_depth': 8, 'classifier__alpha': 0.001} score: 0.7046189546470153
{'classifier__n_estimators': 200, 'classifier__max_depth': 8, 'classifier__alpha': 0.001} score: 0.7069040903593207
{'classifier__n_estimators': 300, 'classifier__max_depth': 8, 'classifier__alpha': 0.001} score: 0.7070501084395611
{'classifier__n_estimators': 400, 'classifier__max_depth': 8, 'classifier

In [0]:
rs.param_distributions

{'classifier__alpha': [0.001],
 'classifier__max_depth': [7, 8],
 'classifier__n_estimators': [100, 200, 300, 400, 500]}

In [0]:
results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__n_estimators,param_classifier__max_depth,param_classifier__alpha,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,21.985280,1.492308,2.553186,0.090431,100,7,0.001,"{'classifier__n_estimators': 100, 'classifier_...",0.702559,0.709877,0.697806,0.703414,0.004965,10,0.963156,0.961104,0.956931,0.960397,0.002590
1,22.110336,2.152032,2.557841,0.061372,200,7,0.001,"{'classifier__n_estimators': 200, 'classifier_...",0.707179,0.713910,0.702577,0.707888,0.004654,2,0.988452,0.986954,0.984431,0.986613,0.001659
2,23.740476,3.408915,2.618038,0.047784,300,7,0.001,"{'classifier__n_estimators': 300, 'classifier_...",0.706343,0.714950,0.705730,0.709008,0.004209,1,0.996100,0.994868,0.994042,0.995003,0.000846
3,27.385717,2.190036,2.675483,0.002884,400,7,0.001,"{'classifier__n_estimators': 400, 'classifier_...",0.705916,0.712842,0.703880,0.707546,0.003836,3,0.998564,0.998043,0.997703,0.998103,0.000354
4,31.818608,1.725086,2.729488,0.062089,500,7,0.001,"{'classifier__n_estimators': 500, 'classifier_...",0.705601,0.711054,0.705358,0.707338,0.002629,4,0.999517,0.999347,0.999235,0.999366,0.000116
5,19.448376,4.130234,2.490289,0.053600,100,8,0.001,"{'classifier__n_estimators': 100, 'classifier_...",0.704271,0.710243,0.699343,0.704619,0.004457,9,0.980928,0.979336,0.975647,0.978637,0.002212
6,22.072066,2.224861,2.576674,0.025767,200,8,0.001,"{'classifier__n_estimators': 200, 'classifier_...",0.703685,0.711758,0.705268,0.706904,0.003493,6,0.994990,0.995281,0.993538,0.994603,0.000762
7,28.053412,2.572536,2.684242,0.032703,300,8,0.001,"{'classifier__n_estimators': 300, 'classifier_...",0.704067,0.711412,0.705670,0.707050,0.003153,5,0.998596,0.998510,0.998222,0.998442,0.000160
8,33.297779,1.939659,2.726955,0.071054,400,8,0.001,"{'classifier__n_estimators': 400, 'classifier_...",0.703459,0.709936,0.706397,0.706597,0.002648,7,0.999641,0.999632,0.999456,0.999576,0.000085
9,39.035829,1.311941,2.359267,0.584672,500,8,0.001,"{'classifier__n_estimators': 500, 'classifier_...",0.703682,0.708525,0.704880,0.705696,0.002060,8,0.999919,0.999912,0.999844,0.999892,0.000034


In [0]:
# make predictions for test data
y_pred = clf.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 84.02%


<h2>1.2 Splitting data into Train and cross validation(or test): Stratified Sampling</h2>

In [0]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

<h2>1.3 Make Data Model Ready: encoding eassay, and project_title</h2>

In [0]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# make sure you featurize train and test data separatly

# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

<h2>1.4 Make Data Model Ready: encoding numerical, categorical features</h2>

In [0]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding 
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# make sure you featurize train and test data separatly

# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

<h2>1.5 Appling Models on different kind of featurization as mentioned in the instructions</h2>

<br>Apply GBDT on different kind of featurization as mentioned in the instructions
<br> For Every model that you work on make sure you do the step 2 and step 3 of instrucations

In [0]:
# please write all the code with proper documentation, and proper titles for each subsection
# go through documentations and blogs before you start coding
# first figure out what to do, and then think about how to do.
# reading and understanding error messages will be very much helpfull in debugging your code
# when you plot any graph make sure you use 
    # a. Title, that describes your plot, this will be very helpful to the reader
    # b. Legends if needed
    # c. X-axis label
    # d. Y-axis label

<h1>3. Summary</h1>

<br> as mentioned in the step 4 of instructions